# Functional programming style

## Applying function to iterables

Consider the following lists as running examples.

In [4]:
import random
l1 = list(range(10))
l2 = random.choices(range(3), k=7)
print(l1, l2)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9] [0, 0, 1, 0, 1, 0, 0]


The `map` function can be used to apply a function to any element of an iterable, e.g.,

In [6]:
list(map(str, map(lambda x: x % 2 == 0, l1)))

['True',
 'False',
 'True',
 'False',
 'True',
 'False',
 'True',
 'False',
 'True',
 'False']

Note that the result of `map` is not a list, but rather an iterator, even when it is appllied to a list.

The `filter` function applies a Boolean predicate to each element of an iterable, and retains only those elements for which the predicate evaluates to `True`.  In the example below, only the even elements will be returned.

In [7]:
list(filter(lambda x: x % 2 == 0, l1))

[0, 2, 4, 6, 8]

Python has quite number of reduction functions such as `sum`, `max`, and `min`, however, using the `reduce` function in `functools` we can easily write our own.

In [15]:
from collections import Counter
from functools import reduce

In [22]:
def incr(c, x):
    c[x] += 1
    return c
reduce(incr, l2, Counter())

Counter({0: 3, 1: 2, 2: 2})

However, note that there is no performance advantage in using `reduce` for this particular case:

In [23]:
l_long = random.choices(range(5), k=1_000_000)

In [24]:
%timeit reduce(incr, l_long, Counter())

1.42 s ± 65.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
%%timeit
counter = Counter()
for x in l_long:
    counter[x] += 1

1.04 s ± 87.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Higher order functions

A new function can be created out of an existing function using `partial` to fix values for one or more arguments of the original function.

In [16]:
from functools import partial

In [25]:
def line_function(x, slope, intercept):
    return slope*x + intercept

In [26]:
line1 = partial(line_function, slope=2.0, intercept=3.0)

In [27]:
line1(0.0), line1(1.0)

(3.0, 5.0)

More generally, we can define a closure, i.e., a function that returns another function.

In [28]:
def create_line(slope, intercept):
    def line(x):
        return slope*x + intercept
    return line

In [29]:
line2 = create_line(2.0, 3.0)

In [30]:
line2(0.0), line2(1.0)

(3.0, 5.0)

## Coroutines

Coroutines are special functions that have at least one, but possibly multiple entry points for suspending and resuming execution, while retaining state between suspend and resume.

In the function below, the function's execution will be suspended at the `yield` expression, and will be resumed when the funciton is called with the number of the next floor for the elevator. This is accomplished using the `send` method defined on coroutine functions.

In [1]:
def create_elevator(highest_floor):
    def elevator():
        floor = 0
        while True:
            next_floor = (yield f'current floor: {floor}')
            if next_floor is not None:
                if 0 <= next_floor <= highest_floor:
                    floor = next_floor
                else:
                    print(f'no such floor: {next_floor}')
            else:
                break
    my_elevator = elevator()
    next(my_elevator)
    return my_elevator    

In [2]:
my_elevator = create_elevator(2)

In [5]:
for floor in random.choices([-1, 0, 1, 2, 3], k=10):
    print(f'press {floor}: {my_elevator.send(floor)}')

press 0: current floor: 0
press 1: current floor: 1
press 2: current floor: 2
no such floor: 3
press 3: current floor: 2
no such floor: -1
press -1: current floor: 2
no such floor: -1
press -1: current floor: 2
press 1: current floor: 1
press 1: current floor: 1
no such floor: 3
press 3: current floor: 1
press 1: current floor: 1
